# Representación de las palabras

Como hemos visto en el tema de redes convolucionales, las imágenes se representan como la cantidad de rojo, verde y azul de cada pixel. Esa cantidad es un número que varía de entre 0 y 255. Es decir, para representar una imagen necesitamos hacerlo mediante números.

Con el lenguaje pasa igual, para poder procesarlo y realizar predicciones o generar texto, necesitamos poder representarlo mediante números. Vamos a ver varias formas de representar el lenguaje mediante números: `encoding ordinal`, `one-hot encoding` y `word embedding`

## Encoding ordinal

Esta es la manera más básica de representar numéricamente un lenguaje, y consiste en asignar un número a cada palabra, por ejemplo, podemos decir que gato lo representaremos con un 1, perro con un 2, mesa con un 3, ...

## One-hot encoding

Aunque el encoding ordinal nos resuelve el problema tiene varios problemas

 * En el ejemplo que hemos dado se puede asumir que mesa (3) equivale a gato (1) + perro (2), lo cual no tiene nada que ver. En el lenguaje existen relaciones entre las palabras, por ejemplo perro y perra tienen mucha relación, por lo que es necesario un sistema de codificación mejor
 * Importancia de las palabras. Las palabras van a entrar a redes neuronales, que como hemos visto se componene de capas, con unos pesos, mediante las cuales se realizan unas operaciones matemáticas. Por lo que puede llegar a pasar que la red le de más importancia a las palabras con un número mayor

Debido a esto, se pensó en una alternativa, el one-hot encodding. Aquí lo que se hace es crear vectores de tamaño N, donde cada palabra corresponderá a un vector de todo ceros, menos un uno en una posición determinada. En el ejemplo de antes gato correspondería al vector `[1 0 0 ... 0]`, perro al vector `[0 1 0 ... 0]`, mesa al vector `[0 0 1 ... 0]`, ...

Haciendo esta codificación arreglamos los dos problemas que hemos contado del encoding ordinal y además añadimos una ventaja, como las redes neuronales realizan operaciones matriciales, si a una matriz le multiplicas por un vector de todo ceros, menos un uno en una posición, lo que estás haciendo es que el resultado de la operación es obtener la columna o la fila correspondiente a la posición del 1

![one-hot encodding matrix multiplication](Imagenes/one_hot_encoding_matrix_multiplication.png)

Esto es muy útil si en redes neuronales quieres una fila o una columna de una matriz, porque al no obtener la fila o la columna haciendo slicing, es decir, `matriz[i]`, sino que se obtiene mediante una multiplicación, esta operación es deribable y por tanto se podría añadir al algoritmo del descenso del gradiente

## Word embedding

El one-hot encoding está muy bien, pero crea un nuevo problema, y es que en un lenguaje de un millón de palabras, necesitaríamos que cada palabra se codificase con vectores de 999.999 ceros y 1 solo uno. Esto a la hora de hacer operaciones matriciales no es muy eficiente y hace que las redes y datos ocupen mucha memoria

Para solucionar esto se empezó a usar el hot encoding a secas o también llamado word embedding, que consiste en seguir representando cada palabra en vectores, pero ahora cada vector tendrá un tamaño fijo, de momento digamos N, y en el que cada uno de sus componentes puede tener cualquier valor.

Esto resuelve el problema de la memoria y la ineficiencia en las operaciones matriciales

Además crea una nueva característica, ya que al representar las palabras en vectores de N dimensiones, en realidad lo que estamos haciendo es representar las palabras en un espacio N dimensional. Si esto te suena a muy complicado no te preocupes, que ahora te lo cuento de otra manera , ya verás que sencillo es y cómo lo vas a entender

Supongamos que en vez de ser un espacio de N dimensiones, tenemos un espacio de 2 dimensiones, alto y ancho, pues podríamos representar las palabras de esta manera

![word embedding 2 dimmension](Imagenes/word_embedding_2_dimmension.png)

Como puedes ver todas las palabras que tienen semejanza están juntas. Pues ahora supón que en vez de 2 dimensiones tenemos 3, alto, ancho y profundo, podríamos representar las palabras de esta manera

![word embedding 3 dimmension](Imagenes/word_embedding_3_dimmension.png)

Ahora las palabras siguen estando juntas por semejanza, pero tenemos una dimensión más para poder hacer más grupos.

Como en el lenguaje tenemos muchisimas palabras no nos vale con 3 dimensiones, por lo que tenemos que hacerlo con muchas más. El modelo más grande de word embeding de Bert es de 1024 dimensiones, mientras que el modelo más grande de word embeding de GPT3 es de 4096 dimensiones

Esta posibilidad de poder representar las palabras en grupos por semejanza también nos da otra ventaja, y es la relación entre palabras. Si al vector que representa la palabra `rey` le restas el vector que representa la palabra `hombre` y le sumas el vector que representa la palabra `mujer` obtienes un vector muy parecido al que representa la palabra `reina`.

Como hemos dicho, en el lenguaje existe una gran relación entre las palabras, lo cual va a ser muy importante para entender frases y además es uno de los mecanismos más importantes de los transformers, el de atención, de hecho el paper de los transformers se llama `Attention is all you need` (`Atención es todo lo que necesitas`). Así que gracias a esta forma de representar las palabras podemos prestar atención a esta relación entre palabras

Este tipo de representación de las palabras es como se hace en los transformers, y es el primer bloque de la arquitectura

![word embedding - transformer architecture](Imagenes/transformer_architecture_model_input_embedding.png)

Como hemos explicado, este bloque lo que hace es recibir palabras de un lenguaje y las convierte a vectores de manera que podamos trabajar con ellos

### Como se crean los word embeddings

Hemos explicado cómo se representan muy bien las palabras gracias a los word embedding, hemos mostrado como las palabras con semejanzas están representadas juntas y hemos hablado de los word embeddings de Bert y GPT3. Pero no hemos explicado cómo se crean, es decir, cómo se decide qué valor tiene cada item del vector para cada palabra. Obviamente eso no lo hace ninguna persona

Cuando se entrena un transformer los pesos que están en esta capa se modifican de manera que el transformer consiga el menor `loss` posible. Supongamos el uso inicial que se pensó para los transformers, traducción. Al transformer le entraban palabras, por un lado la capa de `input embedding` generaba vectores con esas palabras y por otro lado el resto de capas hacen su trabajo, que veremos más adelante. Al principio, como los pesos de la capa de `input embedding` son generados aleatoriamente los vectores generados que representan las palabras no tienen ningún sentido. Pero a medida que se va entrenando y se van modificando los pesos mediante el descenso del gradiente, los vectores generados que representan las palabras van teniendo más sentido. Gracias a los mecanismos de atención (que veremos más adelante) las palabras que tienen relación unas con otras se irán posicionando juntas en el espacio vectorial.

### Implementación

#### Implementación con Pytorch

Pytorch tiene un módulo de embedding (`nn.Embedding`) para facilitarnos el trabajo, veamos cómo funciona

Primero vamos a crear el embedding, para ello tenemos que indicarle cuantas palabras va a tener nuestro modelo, primero vamos a empezar con un modelo de 10 palabras. 

Por otro lado le tenemos que decir las dimensiones que va a tener el embedding, es decir, las dimensiones del nuevo espacio vectorial que se va a crear para convertir cada palabra en un vector, o si lo prefieres, las dimensiones del vector en el que se va a convertir la palabra. Vamos a empezar con vectores de 3 dimensiones

In [34]:
import torch
from torch import nn

vocab_size = 10
embedding_size = 3
embedding = nn.Embedding(vocab_size, embedding_size)

Ahora tenemos que crear las entradas para nuestro embedding. Aquí vamos a hacer algo similar a lo que contamos en el encoding ordinal, sean las que sean las palabras de nuestro vocabulario, las vamos a etiquetar de la 0 a la 9

Puedes pensar que por qué hemos vuelto al encoding ordinal, pero no estamos haciendo exactamente eso. Lo que vamos a hacer es darle un índice a cada palabra de nuestro vocabulario, pero el embedding que acabamos de implementar, va a crear vectores para cada una de esas palabras. Esos vectores tendrán todas las ventajas que hemos explicado del word embedding

Así que vamos a crear una entrada para nuestro embedding. Vamos a pensar 10 palabras, por ejemplo `lunes`, `martes`, `miércoles`, `jueves`, `viernes`, `sábado`, `domingo`, `día`, `semana` y `mes`, que tendrán estos índices

|palabra|índice|
|-------|------|
|lunes|0|
|martes|1|
|miércoles|2|
|jueves|3|
|viernes|4|
|sábado|5|
|domingo|6|
|día|7|
|semana|8|
|mes|9|

Así que si a nuestro embedding le queremos introducir la palabra jueves, por ejemplo, lo que le tenemos que hacer es decirle que queremos meterle la palabra 3 de nuestro vocabulario.

Como requisito Pytorch nos pide que lo transformemos a un `LongTensor`

In [52]:
input = torch.LongTensor([3])

Ahora se la introducimos a nuestro embeddin

In [55]:
output = embedding(input)
output, output.shape

(tensor([[ 1.6763, -0.9649, -2.4252]], grad_fn=<EmbeddingBackward0>),
 torch.Size([1, 3]))

Vemos que la palabra 3 de nuestro vocabulario se corresponde con el vector `[ 1.6763, -0.9649, -2.4252]`

Es importante darse cuenta que como hemos definido nuestro vocabulario de 10 palabras, no podemos decirle a nuestro embedding que cree un vector de una palabra cuyo índice sea mayor que 9

In [56]:
input = torch.LongTensor([10])
output = embedding(input)

IndexError: index out of range in self

Obtenemos un error

Podemos ver cómo serían los vectores de todas las palabras de nuestro vocabulario

In [57]:
input = torch.LongTensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
output = embedding(input)
output, output.shape

(tensor([[-1.3954,  1.6954,  2.6419],
         [ 1.1381,  1.8721,  0.2285],
         [-0.2601, -1.9544, -1.0417],
         [ 1.6763, -0.9649, -2.4252],
         [ 0.2824, -0.0044,  0.3790],
         [-0.4283, -1.0143, -1.0585],
         [-0.1005,  0.4807, -1.5059],
         [ 1.1046, -1.3702,  0.3930],
         [-0.8880, -0.3066, -0.8708],
         [-0.2623,  2.2512, -0.0614]], grad_fn=<EmbeddingBackward0>),
 torch.Size([10, 3]))

Hemos obtenido todos los posibles vectores de nuestro vocabulario. Este sería el espacio vectorial

El módulo `nn.Embedding` de Pytorch lo que hace es crear una matriz de manera que al introducirle una palabra nos genere un vector de tamaño 3 (en este ejemplo)

In [58]:
embedding.weight.shape

torch.Size([10, 3])

Si no cambiamos el embedding, siempre que le introduzcamos la misma palabra a nuestro embedding nos devolverá el siguiente vector

In [59]:
input = torch.LongTensor([0])
output = embedding(input)
print(output)
input = torch.LongTensor([0])
output = embedding(input)
print(output)
input = torch.LongTensor([0])
output = embedding(input)
print(output)

tensor([[-1.3954,  1.6954,  2.6419]], grad_fn=<EmbeddingBackward0>)
tensor([[-1.3954,  1.6954,  2.6419]], grad_fn=<EmbeddingBackward0>)
tensor([[-1.3954,  1.6954,  2.6419]], grad_fn=<EmbeddingBackward0>)


Como vemos hemos obtenido siempre el mismo vector

Sin embargo, si inicializamos de nuevo el embedding obtendremos diferentes vectores a la salida para una misma palabra

In [60]:
embedding = nn.Embedding(vocab_size, embedding_size)
input = torch.LongTensor([0])
output = embedding(input)
print(output)
embedding = nn.Embedding(vocab_size, embedding_size)
input = torch.LongTensor([0])
output = embedding(input)
print(output)
embedding = nn.Embedding(vocab_size, embedding_size)
input = torch.LongTensor([0])
output = embedding(input)
print(output)

tensor([[ 0.1781, -0.1437, -1.3322]], grad_fn=<EmbeddingBackward0>)
tensor([[-0.0030,  0.1196,  2.0875]], grad_fn=<EmbeddingBackward0>)
tensor([[-0.0117,  1.6866, -0.3323]], grad_fn=<EmbeddingBackward0>)


Ahora hemos obtenido diferentes vectores. ¿Esto que quiere decir? Esto quiere decir que Pytorch ha creado esta matriz con valores aleatorios, por lo que los vectores que nos genera nuestro embedding no tiene ningún sentido. Sin embargo, a la hora de entrenar el transformer, los pesos del embedding se irán modificando de manera que habrá un momento que tengamos un espacio vectorial, en el que como hemos explicado, todas las palabras similares estén juntas

Por último, como la idea final es crearnos nuestro propio transformer y entrenarlo, vamos a crearnos la clase `Embedding` para usarla en el futuro

In [61]:
from torch import nn

class Embedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, x):
        return self.embedding(x)

Ahora podemos inicializarla con los valores de tamaño de vocabulario y tamaño de embedding de antes y ver que obtendremos los mismos resultados

In [62]:
embedding = Embedding(vocab_size, embedding_size)
input = torch.LongTensor([0])
output = embedding(input)
print(output)

tensor([[ 0.9370,  0.3952, -1.7952]], grad_fn=<EmbeddingBackward0>)


#### Implementación con `huggingface`

La empresa [huggingface](https://huggingface.co) se hizo muy popular gracias a su librería [transformers](https://huggingface.co/docs/transformers), ya que hizo que fuese muy sencillo usarlos gracias a su API. Ahora no solo podemos usar transformers, sino casi cualquier modelo, ya que se autodefinen como el GitHub del deep learning, así que casi todos los nuevos modelos se publican también en huggingface, donde los puedes usar con su librería, ver su documentación e incluso probar online gracias a sus `spaces`

En este curso no vamos a explicar hugginface, pero es bueno que la conozcas, ya que es un muy buen recurso

Vamos a usar el `input embedding` de Bert para ver cómo representa las palabras

Primero vamos a importar las librerías necesarias

In [63]:
from transformers import BertTokenizer, BertModel
import torch

/home/wallabot/miniconda3/envs/cursopytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cargamos el modelo Bert

In [64]:
model = BertModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Seguramente obtendrás un warning como yo, ya que estamos importando los pesos de `bert-base-multilingual-cased` que fue entrenado con más capas de las que aparecen en el modelo `BertModel`. Si fuésemos a usar el modelo para lo que fue entrenado y dio como resultado los pesos `bert-base-multilingual-cased` deberíamos importar las capas extra, pero como no es nuestro caso podemos ignorar el warning

Ahora su tokenizador

In [65]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Creamos el token de la palabra `hola`

In [66]:
input_ids = tokenizer.encode("hola", add_special_tokens=True)
input_ids

[101, 110516, 10113, 102]

Primero vamos a analizar los `ids` o tokens. Por cómo está implementado BERT siempre espera que cada frase empieze con el token `[CLS]` (`101`) y que termine con el token `[SEP]` (`102`), por lo que el tokenizador de BERT los añade automáticamente

Por otro lado los tokens `110516` y `10113` son los tokens generados por dividir la palabra `hola` en dos subpalabras, que son

In [69]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
tokens

['[CLS]', 'hol', '##a', '[SEP]']

BERT ha dividido la palabra `hola` en `hol` y `##a`. Esto es porque espera palabras en inglés y al no ser `hola` una palabra inglesa la ha intentado subdividir en dos palabras más sencillas del ingleś. Así que en vez de introducirle `hola` vamos a ver qué pasa si le introducimos `hello`

In [74]:
input_ids = tokenizer.encode("hello", add_special_tokens=True)
print(input_ids)
tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(tokens)

[101, 61694, 10133, 102]
['[CLS]', 'hell', '##o', '[SEP]']


También lo divide en subpalabras más sencillas.

Esto es algo que no hemos explicado, porque no corresponde al ningún módulo del modelo transformer, sino a una etapa del preprocesado del lenguaje.

Cuando se crea el vocabulario que se quiere que tenga nuestro modelo, se hacen descomposiciones de palabras en otras más sencillas, a estas se les llaman `tokens`

De modo que no habrá un token `perro` y otro `perra` sino que habrá un token `perr`, otro `##o` y otro `##a` (las dobles almohadillas indican que son el final de una palabra). Puedes pensar, pero `perro` y `perra` son dos tokens y `perr`, `##o` y `##a` son tres tokens, y es verdad, pero ahora podemos añadir el token `herman` y juntarlo con los tokens ya creados `##o` y `##a`, y añadir el token `cuñad` y volver a juntarlo con los tokens ya creados `##o` y `##a`. De esta manera hemos pasado de tener 6 tokens `perro`, `perra`, `hermano`, `hermana`, `cuñado` y `cuñada` a solo 5 con `perr`, `herman`, `cuñad`, `##o` y `##a`. Hemos ahorrado un token, y a medida que añadamos más palabras terminadas en `o` o `a` seguiremos reduciendo el número de tokens.

Esta técnica y otras de dividir las palabras en otras más sencillas se llama tokenización, lo que produce un vocabulario más pequeño y así poder usar modelos más pequeños

Más adelante entenderás esto, pero el modelo Bert solo contiene la parte de codificación del transformer, por lo que para obtener el embedding de la palabra `hola` lo que hacemos es pasarla por el modelo entero. Primero convertimos su token a un vector y luego hacemos inferencia

In [79]:
input_ids = tokenizer.encode("hello", add_special_tokens=True)
input_ids_tensor = torch.tensor([input_ids])
with torch.no_grad():
    outputs = model(input_ids_tensor)

Obtenemos el embedding

In [80]:
# El primer elemento de la salida del modelo son los embeddings para cada token.
# Tomamos el segundo token (índice 1) ya que el primer token es [CLS]
hola_embedding = outputs[0][0][1]

type(hola_embedding), hola_embedding.shape

(torch.Tensor, torch.Size([768]))

Como vemos obtenemos un tensor de tamaño 768, esto quiere decir que el modelo Bert que hemos usado tiene un word embedding de tamaño 768, es decir, cada palabra estará representada por un vector de 768 valores distintos

### Palabras similares en la misma zona del espacio vectorial

Vamos a ver el ejemplo que hemos contado antes, en el que al embedding de la palabra `rey` le restamos el embedding de la palabra `hombre` y le sumamos el embedding de la palabra `mujer` y comparamos el resultado con el embedding de la palabra `reina`. Si son similares es porque como hemos dicho, las palabras similares se encuentran en las mismas zonas del espacio vectorial, por lo que restando y sumando nos llevamos la palabra `rey` a la zona de la palabra `reina`.

Para ver la similitud entre el resultado y el embedding de la palabra reina vamos a utilizar la función `cosine_similarity` que es como se comprueba la similitud entre palabras en un espacio vectorial. Más adelante, cuando expliquemos los bloques de atención explicaremos el por qué de esto. De momento solo quédate, que cuanto más cercano a 1, mas similares serán las palabras

Lo hacemos con el embedding del modelo BERT porque ya está entrenado

Primero importamos las librerías necesarias

In [11]:
from transformers import BertTokenizer, BertModel
import torch
from torch.nn.functional import cosine_similarity

Ahora cargamos el tockenizador de bert para obtener los tokens de las palabras

In [12]:
# Carga el pre-trained BERT
model = BertModel.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Como hemos explicado antes puede salir un warning pero no nos afecta

Ahora creamos una lista con las palabras de las que queremos obtener el embedding

In [13]:
# Lista de palabras
palabras = ["rey", "reina", "hombre", "mujer"]

Creamos un dicionario donde vamos a guardar los embedings de las palabras

In [14]:
# Diccionario para guardar los embeddings
embeddings = {}

Obtenemos el embedding de cada palabra

In [15]:
for palabra in palabras:
    # Codifica la palabra en tokens y agrega los tokens especiales [CLS] y [SEP]
    input_ids = tokenizer.encode(palabra, add_special_tokens=True)

    # Convierte los IDs a tensores y pasa por el modelo para obtener los embeddings
    input_ids_tensor = torch.tensor([input_ids])
    with torch.no_grad():
        outputs = model(input_ids_tensor)

    # El primer elemento de la salida del modelo son los embeddings para cada token.
    # Tomamos el segundo token (índice 1) ya que el primer token es [CLS]
    embeddings[palabra] = outputs[0][0][1]

Realizamos la operación de `rey` menos `hombre` más `mujer`

In [16]:
# Realiza la operación rey - hombre + mujer
resultado = embeddings["rey"] - embeddings["hombre"] + embeddings["mujer"]

Y por último calculamos la similitud entre el embedding del cálculo y el embedding de la palabra `reina`

In [17]:
# Calcula la similitud coseno entre el resultado y la palabra "reina"
similitud = cosine_similarity(resultado.unsqueeze(0), embeddings["reina"].unsqueeze(0))
print(f"La similitud coseno entre 'rey - hombre + mujer' y 'reina' es: {similitud.item()}")

La similitud coseno entre 'rey - hombre + mujer' y 'reina' es: 0.728291928768158


Como hemos dicho, ya explicaremos por qué se usa el coseno para calcular la similitud, pero como el resultado es cercano a 1 podemos decir que el embedding resultante de `hombre` - `hombre` + `mujer` es muy similar al embedding de la palabra `reina`

La capa de input embedding del modelo Bert parece que está muy bien conseguida

### Transfer learning

Como hemos explicado, a la hora de entrenar un transformer se entrena también la capa de `input embedding`. Pero en según que casos podemos usar el `input embedding` de un modelo grande de lenguaje, como en este caso el de Bert, o de otros modelos más grandes, cuyo espacio vectorial sea más grande que el que hemos utilizado ahora, o que esté especializado en el dominio en el que queramos trabajar, etc.

Una vez tenemos el embedding de otro modelo, podemos entrenar el transformer entero, incluido el embedding, o congelar el embedding (en los temas anteriores hemos visto que se puede hacer esto para que no cambien sus pesos) y así solo entrenaremos el resto del transformer, pero no su embedding, aprovechando que usamos uno ya entrenado

Aquí ya depende del caso de uso, si nos vale uno preentrenado genérico podemo congelarlo y entrenar el resto. Pero si lo que queremos es tener un transformer especializado en un tema muy concreto, por ejemplo, un transformer médico, deberíamos entrenar la capa de embedding también ya que puede que haya palabras técnicas que no estuvieran en el vocabulario con el que se entrenó el embedding y por tanto no sepa cómo crear vectores para esas palabras